In [3]:
%matplotlib inline
import ast
from bs4 import BeautifulSoup
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
from numpy.polynomial.polynomial import polyfit
import pandas as pd
import re
import requests
from scipy import stats
import seaborn as sns
import sqlite3
from sqlite3 import Error
import time
import datetime
from config import api_key_tmdb

In [12]:
#reading csv file and loading it as a dataframe
imdb_movies= pd.read_csv("imdb_movies_data.csv")
tmdb_cast = pd.read_csv("tmdb_movies_cast_df.csv" , engine='python')
tmdb_movies = pd.read_csv("tmdb_movies_df.csv", engine='python')

In [24]:
imdb_movies.head()

,Unnamed: 0,Unnamed: 0.1,title,genre,imdb_score,meta_score,imdb_id,year
0,0,0,Spider-Man: Into the Spider-Verse,"['Animation', ' Action', ' Adventure']",8.5,87,tt4633694,2018
1,1,1,Aquaman,"['Action', ' Adventure', ' Fantasy']",7.2,55,tt1477834,2018
2,2,2,Avengers: Infinity War,"['Action', ' Adventure', ' Sci-Fi']",8.5,68,tt4154756,2018
3,3,3,Bumblebee,"['Action', ' Adventure', ' Sci-Fi']",7.0,66,tt4701182,2018
4,4,4,Dragged Across Concrete,"['Action', ' Crime', ' Drama']",7.4,63,tt6491178,2018


In [25]:
tmdb_cast.head()
del_col_list = ['overview', 'tagline', 'vote_average', 'vote_count', 'runtime', 'popularity', 'production com']

,id,title,belongs_to_collection,budget,genres,imdb_id,overview,release_date,revenue,tagline,vote_average,vote_count,runtime,popularity,production_companies,director_name,director_gender,leading_actor_name,supporting_actor_name
0,324857,Spider-Man: Into the Spider-Verse,Spider-Man: Into the Spider-Verse Collection,90000000.0,"['Action', 'Adventure', 'Animation', 'Science ...",tt4633694,Miles Morales is juggling his life between bei...,2018-12-07,362823577.0,More Than One Wears the Mask,8.5,2899.0,117.0,115.129,Columbia Pictures,NaN,NaN,NaN,NaN
1,297802,Aquaman,Aquaman Collection,160000000.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",tt1477834,Once home to the most advanced civilization on...,2018-12-07,1143689193.0,Home Is Calling,6.8,4885.0,144.0,129.402,DC Comics,NaN,NaN,NaN,NaN
2,299536,Avengers: Infinity War,The Avengers Collection,300000000.0,"['Adventure', 'Action', 'Fantasy']",tt4154756,As the Avengers and their allies have continue...,2018-04-25,2046239637.0,An entire universe. Once and for all.,8.3,12297.0,149.0,129.988,Marvel Studios,NaN,NaN,NaN,NaN
3,424783,Bumblebee,Transformers Collection,135000000.0,"['Action', 'Adventure', 'Science Fiction']",tt4701182,"On the run in the year 1987, Bumblebee finds r...",2018-12-15,420361283.0,Every Hero Has a Beginning,6.5,1586.0,114.0,226.241,Hasbro,NaN,NaN,NaN,NaN
4,438674,Dragged Across Concrete,NaN,15000000.0,"['Drama', 'Crime', 'Action', 'Thriller']",tt6491178,"Two policemen, one an old-timer, the other his...",2019-02-21,NaN,Those who can't earn a living must find anothe...,6.9,23.0,159.0,34.986,Cinestate,NaN,NaN,NaN,NaN


In [27]:
tmdb_movies.head()

,Unnamed: 0,id,title,belongs_to_collection,budget,genres,imdb_id,overview,release_date,revenue,tagline,vote_average,vote_count,runtime,popularity,production_companies
0,0,324857,Spider-Man: Into the Spider-Verse,Spider-Man: Into the Spider-Verse Collection,90000000.0,"['Action', 'Adventure', 'Animation', 'Science ...",tt4633694,Miles Morales is juggling his life between bei...,2018-12-07,3.628236e+08,More Than One Wears the Mask,8.5,2899.0,117.0,115.129,Columbia Pictures
1,1,297802,Aquaman,Aquaman Collection,160000000.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",tt1477834,Once home to the most advanced civilization on...,2018-12-07,1.143689e+09,Home Is Calling,6.8,4885.0,144.0,129.402,DC Comics
2,2,299536,Avengers: Infinity War,The Avengers Collection,300000000.0,"['Adventure', 'Action', 'Fantasy']",tt4154756,As the Avengers and their allies have continue...,2018-04-25,2.046240e+09,An entire universe. Once and for all.,8.3,12297.0,149.0,129.988,Marvel Studios
3,3,424783,Bumblebee,Transformers Collection,135000000.0,"['Action', 'Adventure', 'Science Fiction']",tt4701182,"On the run in the year 1987, Bumblebee finds r...",2018-12-15,4.203613e+08,Every Hero Has a Beginning,6.5,1586.0,114.0,226.241,Hasbro
4,4,438674,Dragged Across Concrete,NaN,15000000.0,"['Drama', 'Crime', 'Action', 'Thriller']",tt6491178,"Two policemen, one an old-timer, the other his...",2019-02-21,NaN,Those who can't earn a living must find anothe...,6.9,23.0,159.0,34.986,Cinestate


In [4]:
# Create a connection object using the connect function 
# To establish a connection with "movies.db"
connection = sqlite3.connect("movies.db")
# Create a cursor object by calling the cursor method
cursor = connection.cursor()

In [20]:
#creating tables into movies database
imdb_movies.to_sql("IMDB_movies", con=conn)
tmdb_cast.to_sql("TMDB_cast", con=conn)
tmdb_movies.to_sql("TMDB_movies", con=conn)

/Users/fhel/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)
/Users/fhel/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


In [23]:
c.execute('''select * from IMDB_movies''').fetchall()

[(0,
  0,
  0,
  'Spider-Man: Into the Spider-Verse',
  "['Animation', ' Action', ' Adventure']",
  '8.5',
  '87',
  'tt4633694',
  2018),
 (1,
  1,
  1,
  'Aquaman',
  "['Action', ' Adventure', ' Fantasy']",
  '7.2',
  '55',
  'tt1477834',
  2018),
 (2,
  2,
  2,
  'Avengers: Infinity War',
  "['Action', ' Adventure', ' Sci-Fi']",
  '8.5',
  '68',
  'tt4154756',
  2018),
 (3,
  3,
  3,
  'Bumblebee',
  "['Action', ' Adventure', ' Sci-Fi']",
  '7.0',
  '66',
  'tt4701182',
  2018),
 (4,
  4,
  4,
  'Dragged Across Concrete',
  "['Action', ' Crime', ' Drama']",
  '7.4',
  '63',
  'tt6491178',
  2018),
 (5,
  5,
  5,
  'Deadpool 2',
  "['Action', ' Adventure', ' Comedy']",
  '7.8',
  '66',
  'tt5463162',
  2018),
 (6,
  6,
  6,
  'Mortal Engines',
  "['Action', ' Adventure', ' Fantasy']",
  '6.2',
  '44',
  'tt1571234',
  2018),
 (7,
  7,
  7,
  'Black Panther',
  "['Action', ' Adventure', ' Sci-Fi']",
  '7.3',
  '88',
  'tt1825683',
  2018),
 (8,
  8,
  8,
  'Thor: Ragnarok',
  "['Actio

In [5]:
#Searches how many movies have a metascore >90 in the span of a decade.  This is 2010-2018
cursor.execute('''select title, meta_score, imdb_id from imdb_movies where meta_score >90 and year between 2010 and 2019;''').fetchall()
#Count is 30, AVG metascore is 93.9


[('Dunkirk', '94', 'tt5013056'),
 ('Roma', '96', 'tt6155172'),
 ('Call Me by Your Name', '93', 'tt5726616'),
 ('La La Land', '93', 'tt3783958'),
 ('Shoplifters', '93', 'tt8075192'),
 ('Dunkirk', '94', 'tt5013056'),
 ('Moonlight', '99', 'tt4975722'),
 ('12 Years a Slave', '96', 'tt2024544'),
 ('Spotlight', '93', 'tt1895587'),
 ('Lady Bird', '94', 'tt4925292'),
 ('Zero Dark Thirty', '95', 'tt1790885'),
 ('Inside Out', '94', 'tt2096673'),
 ('The Social Network', '95', 'tt1285016'),
 ('Manchester by the Sea', '96', 'tt4034228'),
 ('Gravity', '96', 'tt1454468'),
 ('The Florida Project', '92', 'tt5649144'),
 ('Carol', '95', 'tt2402927'),
 ('Inside Llewyn Davis', '93', 'tt2042568'),
 ('A Separation', '95', 'tt1832382'),
 ('The Rider', '92', 'tt6217608'),
 ('Before Midnight', '94', 'tt2209418'),
 ('Ida', '91', 'tt2718492'),
 ('Amour', '94', 'tt1602620'),
 ('45 Years', '94', 'tt3544082'),
 ('Toni Erdmann', '93', 'tt4048272'),
 ('Leviathan', '92', 'tt2802154'),
 ('Mr. Turner', '94', 'tt2473794')

In [6]:
#find the avg of the meta score from this data
cursor.execute('''select AVG(meta_score) from imdb_movies where meta_score >90 and year between 2010 and 2019;''').fetchall()


[(93.9,)]

In [7]:
#searches Y2K to 2009
cursor.execute('''select title, meta_score, imdb_score from imdb_movies where meta_score >90 and year between 2000 and 2009;''').fetchall()
#Count is 16, avg meta_score is 93.25

[('Crouching Tiger, Hidden Dragon', '93', '7.9'),
 ('The Lord of the Rings: The Fellowship of the Ring', '92', '8.8'),
 ('No Country for Old Men', '91', '8.1'),
 ('The Lord of the Rings: The Return of the King', '94', '8.9'),
 ("Pan's Labyrinth", '98', '8.2'),
 ('Ratatouille', '96', '8.0'),
 ('There Will Be Blood', '93', '8.2'),
 ('The Hurt Locker', '94', '7.6'),
 ('Sideways', '94', '7.5'),
 ('The Queen', '91', '7.3'),
 ('The Diving Bell and the Butterfly', '92', '8.0'),
 ('The Triplets of Belleville', '91', '7.8'),
 ('4 Months, 3 Weeks and 2 Days', '97', '7.9'),
 ('Waltz with Bashir', '91', '8.0'),
 ('Yi Yi', '93', '8.1'),
 ('Werckmeister harmóniák', '92', '8.2')]

In [8]:
#searches 1990-1999
cursor.execute('''select title, meta_score, imdb_score from imdb_movies where meta_score >90 and year between 1990 and 1999;''').fetchall()
#Count 9, avg is 93.111

[('Pulp Fiction', '94', '8.9'),
 ("Schindler's List", '93', '8.9'),
 ('Saving Private Ryan', '91', '8.6'),
 ('Secrets & Lies', '91', '8.0'),
 ('Beau travail', '91', '7.4'),
 ('Reversal of Fortune', '93', '7.3'),
 ('Days of Being Wild', '96', '7.6'),
 ('After Life', '91', '7.7'),
 ('Hoop Dreams', '98', '8.3')]

In [9]:
#searches 1980-1989
cursor.execute('''select title, meta_score, imdb_score from imdb_movies where meta_score >90 and year between 1980 and 1989;''').fetchall()
# Count is 8, AVG is 93.5

[('Ran', '96', '8.2'),
 ('Platoon', '92', '8.1'),
 ('Do the Right Thing', '91', '7.9'),
 ('Amadeus', '93', '8.3'),
 ('The Right Stuff', '91', '7.9'),
 ('My Left Foot', '97', '7.9'),
 ('Ran', '96', '8.2'),
 ('Threads', '92', '8.1')]

In [39]:
#merging databases
movies_df = pd.merge(imdb_movies, tmdb_movies, left_on ='imdb_id', right_on='imdb_id', how='left', suffixes=('', '_y'))
movies_df

,Unnamed: 0,Unnamed: 0.1,title,genre,imdb_score,meta_score,imdb_id,year,Unnamed: 0_y,id,...,genres,overview,release_date,revenue,tagline,vote_average,vote_count,runtime,popularity,production_companies
0,0,0,Spider-Man: Into the Spider-Verse,"['Animation', ' Action', ' Adventure']",8.5,87,tt4633694,2018,0,324857,...,"['Action', 'Adventure', 'Animation', 'Science ...",Miles Morales is juggling his life between bei...,2018-12-07,3.628236e+08,More Than One Wears the Mask,8.5,2899.0,117.0,115.129,Columbia Pictures
1,1,1,Aquaman,"['Action', ' Adventure', ' Fantasy']",7.2,55,tt1477834,2018,1,297802,...,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Once home to the most advanced civilization on...,2018-12-07,1.143689e+09,Home Is Calling,6.8,4885.0,144.0,129.402,DC Comics
2,2,2,Avengers: Infinity War,"['Action', ' Adventure', ' Sci-Fi']",8.5,68,tt4154756,2018,2,299536,...,"['Adventure', 'Action', 'Fantasy']",As the Avengers and their allies have continue...,2018-04-25,2.046240e+09,An entire universe. Once and for all.,8.3,12297.0,149.0,129.988,Marvel Studios
3,3,3,Bumblebee,"['Action', ' Adventure', ' Sci-Fi']",7.0,66,tt4701182,2018,3,424783,...,"['Action', 'Adventure', 'Science Fiction']","On the run in the year 1987, Bumblebee finds r...",2018-12-15,4.203613e+08,Every Hero Has a Beginning,6.5,1586.0,114.0,226.241,Hasbro
4,4,4,Dragged Across Concrete,"['Action', ' Crime', ' Drama']",7.4,63,tt6491178,2018,4,438674,...,"['Drama', 'Crime', 'Action', 'Thriller']","Two policemen, one an old-timer, the other his...",2019-02-21,NaN,Those who can't earn a living must find anothe...,6.9,23.0,159.0,34.986,Cinestate
5,5,5,Deadpool 2,"['Action', ' Adventure', ' Comedy']",7.8,66,tt5463162,2018,5,383498,...,"['Action', 'Comedy', 'Adventure']",Wisecracking mercenary Deadpool battles the ev...,2018-05-15,7.415474e+08,Prepare for the Second Coming.,7.5,7993.0,121.0,42.911,Marvel Entertainment
6,6,6,Mortal Engines,"['Action', ' Adventure', ' Fantasy']",6.2,44,tt1571234,2018,6,428078,...,"['Science Fiction', 'Action', 'Adventure', 'Fa...","Many thousands of years in the future, Earth’s...",2018-11-27,1.042365e+08,Some Scars Never Heal,5.9,1261.0,129.0,70.197,Scholastic Productions
7,7,7,Black Panther,"['Action', ' Adventure', ' Sci-Fi']",7.3,88,tt1825683,2018,7,284054,...,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",King T'Challa returns home from America to the...,2018-02-13,1.346739e+09,Long live the king.,7.4,11333.0,134.0,60.071,Marvel Studios
8,8,8,Thor: Ragnarok,"['Action', ' Adventure', ' Comedy']",7.9,74,tt3501632,2017,8,284053,...,"['Action', 'Adventure', 'Comedy', 'Fantasy', '...",Thor is imprisoned on the other side of the un...,2017-10-25,8.539771e+08,No Hammer. No Problem.,7.5,10535.0,131.0,54.183,Marvel Studios
9,9,9,Jurassic World: Fallen Kingdom,"['Action', ' Adventure', ' Sci-Fi']",6.2,51,tt4881806,2018,9,351286,...,"['Action', 'Adventure', 'Science Fiction']",Three years after the demise of Jurassic World...,2018-06-06,1.303460e+09,The park is gone,6.5,5545.0,129.0,37.211,Amblin Entertainment


In [76]:
all_movies = movies_df = pd.merge(movies_df, tmdb_cast, left_on ='imdb_id', right_on='imdb_id', how='left', suffixes=('', '_y'))
all_movies

,Unnamed: 0,Unnamed: 0.1,title,genre,imdb_score,meta_score,imdb_id,year,Unnamed: 0_y,id,...,tagline_y,vote_average_y,vote_count_y,runtime_y,popularity_y,production_companies_y,director_name_y,director_gender_y,leading_actor_name_y,supporting_actor_name_y
0,0,0,Spider-Man: Into the Spider-Verse,"['Animation', ' Action', ' Adventure']",8.5,87,tt4633694,2018,0,324857,...,More Than One Wears the Mask,8.5,2899.0,117.0,115.129,Columbia Pictures,NaN,NaN,NaN,NaN
1,1,1,Aquaman,"['Action', ' Adventure', ' Fantasy']",7.2,55,tt1477834,2018,1,297802,...,Home Is Calling,6.8,4885.0,144.0,129.402,DC Comics,NaN,NaN,NaN,NaN
2,2,2,Avengers: Infinity War,"['Action', ' Adventure', ' Sci-Fi']",8.5,68,tt4154756,2018,2,299536,...,An entire universe. Once and for all.,8.3,12297.0,149.0,129.988,Marvel Studios,NaN,NaN,NaN,NaN
3,3,3,Bumblebee,"['Action', ' Adventure', ' Sci-Fi']",7.0,66,tt4701182,2018,3,424783,...,Every Hero Has a Beginning,6.5,1586.0,114.0,226.241,Hasbro,NaN,NaN,NaN,NaN
4,4,4,Dragged Across Concrete,"['Action', ' Crime', ' Drama']",7.4,63,tt6491178,2018,4,438674,...,Those who can't earn a living must find anothe...,6.9,23.0,159.0,34.986,Cinestate,NaN,NaN,NaN,NaN
5,5,5,Deadpool 2,"['Action', ' Adventure', ' Comedy']",7.8,66,tt5463162,2018,5,383498,...,Prepare for the Second Coming.,7.5,7993.0,121.0,42.911,Marvel Entertainment,NaN,NaN,NaN,NaN
6,6,6,Mortal Engines,"['Action', ' Adventure', ' Fantasy']",6.2,44,tt1571234,2018,6,428078,...,Some Scars Never Heal,5.9,1261.0,129.0,70.197,Scholastic Productions,NaN,NaN,NaN,NaN
7,7,7,Black Panther,"['Action', ' Adventure', ' Sci-Fi']",7.3,88,tt1825683,2018,7,284054,...,Long live the king.,7.4,11333.0,134.0,60.071,Marvel Studios,NaN,NaN,NaN,NaN
8,8,8,Thor: Ragnarok,"['Action', ' Adventure', ' Comedy']",7.9,74,tt3501632,2017,8,284053,...,No Hammer. No Problem.,7.5,10535.0,131.0,54.183,Marvel Studios,Taika Waititi,2.0,Chris Hemsworth,Tom Hiddleston
9,9,9,Jurassic World: Fallen Kingdom,"['Action', ' Adventure', ' Sci-Fi']",6.2,51,tt4881806,2018,9,351286,...,The park is gone,6.5,5545.0,129.0,37.211,Amblin Entertainment,NaN,NaN,NaN,NaN


In [45]:
drop_column_list = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0_y', 'id', 'title_y', 'runtime','id_y', 'title_y', 'release_date_y', 'revenue_y', 'tagline_y', 'vote_average_y', 'vote_count_y', 'runtime_y', 'popularity_y', 'production_companies_y',
        'id_y', 'title_y', 'belongs_to_collection_y',
       'budget_y', 'genres_y', 'overview_y', 'release_date_y', 'revenue_y',
       'tagline_y', 'vote_average_y', 'vote_count_y', 'runtime_y',
       'popularity_y', 'production_companies_y', 'director_name_y',
       'director_gender_y', 'leading_actor_name_y', 'supporting_actor_name_y']
                    
all_movies = all_movies.drop(drop_column_list, axis=1)
all_movies.head()


,title,genre,imdb_score,meta_score,imdb_id,year,belongs_to_collection,budget,genres,overview,...,revenue,tagline,vote_average,vote_count,popularity,production_companies,director_name,director_gender,leading_actor_name,supporting_actor_name
0,Spider-Man: Into the Spider-Verse,"['Animation', ' Action', ' Adventure']",8.5,87,tt4633694,2018,Spider-Man: Into the Spider-Verse Collection,90000000.0,"['Action', 'Adventure', 'Animation', 'Science ...",Miles Morales is juggling his life between bei...,...,3.628236e+08,More Than One Wears the Mask,8.5,2899.0,115.129,Columbia Pictures,NaN,NaN,NaN,NaN
1,Aquaman,"['Action', ' Adventure', ' Fantasy']",7.2,55,tt1477834,2018,Aquaman Collection,160000000.0,"['Action', 'Adventure', 'Fantasy', 'Science Fi...",Once home to the most advanced civilization on...,...,1.143689e+09,Home Is Calling,6.8,4885.0,129.402,DC Comics,NaN,NaN,NaN,NaN
2,Avengers: Infinity War,"['Action', ' Adventure', ' Sci-Fi']",8.5,68,tt4154756,2018,The Avengers Collection,300000000.0,"['Adventure', 'Action', 'Fantasy']",As the Avengers and their allies have continue...,...,2.046240e+09,An entire universe. Once and for all.,8.3,12297.0,129.988,Marvel Studios,NaN,NaN,NaN,NaN
3,Bumblebee,"['Action', ' Adventure', ' Sci-Fi']",7.0,66,tt4701182,2018,Transformers Collection,135000000.0,"['Action', 'Adventure', 'Science Fiction']","On the run in the year 1987, Bumblebee finds r...",...,4.203613e+08,Every Hero Has a Beginning,6.5,1586.0,226.241,Hasbro,NaN,NaN,NaN,NaN
4,Dragged Across Concrete,"['Action', ' Crime', ' Drama']",7.4,63,tt6491178,2018,NaN,15000000.0,"['Drama', 'Crime', 'Action', 'Thriller']","Two policemen, one an old-timer, the other his...",...,NaN,Those who can't earn a living must find anothe...,6.9,23.0,34.986,Cinestate,NaN,NaN,NaN,NaN


In [54]:
#dropped duplicates
print(all_movies.shape)
all_movies = all_movies.drop_duplicates(keep='first')
print(all_movies.shape)

(18246, 21)
(17368, 21)


In [84]:
#Turning meta_score from a string to an integer
all_movies["meta_score"] = pd.to_numeric(all_movies["meta_score"], downcast = "integer", errors = "coerce")
#was successful except for NAN values

In [83]:
#dropped nan values
all_movies.dropna(subset=['meta_score'], inplace=True)

In [93]:
for i in all_movies['meta_score']:
    int(i)
    print(type(i))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


ValueError: cannot convert float NaN to integer

In [86]:
len(all_movies)

18274

In [106]:
high_meta_movies = all_movies['title'][all_movies['meta_score'] >= 90]
high_meta_movies

43                                      Mad Max: Fury Road
76                                                 Dunkirk
88                      Star Wars: Episode IV - A New Hope
96                                         The Incredibles
508                         Crouching Tiger, Hidden Dragon
719                                            Dirty Harry
772                                         The Wild Bunch
850                                  The French Connection
948                                                    Ran
1269                          The Adventures of Robin Hood
6359                                            Dragon Inn
9026                                         The Favourite
9078                                          Pulp Fiction
9118                                                  Roma
9128                                         The Godfather
9157                                              Cold War
9167     The Lord of the Rings: The Fellowship of the Ri

In [ ]:
high_meta